# Tipos de Dados, Marcas Gráficas e Canais de Codificação Visual

A visualização representa a informação usando uma coleção de _marcas gráficas_ (barras, linhas, pontos, etc.). Os atributos de uma marca - como sua posição, formato, tamanho, ou cor - servem como _canais_ através dos quais podemos codificar valores de dados subjacentes.

Com uma estrutura de tipos de dados, marcas e canais de codificação, podemos criar de forma concisa uma grande variedade de visualizações. Nesse notebook, exploramos cada um desses elementos e mostramos como usá-los para criar gráficos estatísticos customizados.

_Esse notebook é parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._


In [ ]:
import pandas as pd
import altair as alt

: 

##  Dados de Desenvolvimento Global (José)
Vamos visualizar dados de saúde global e população para vários países, no período de 1955 à 2005. Os dados foram coletados pela [Fundação Gapminder](https://www.gapminder.org/) e compartilhados na [famosa palestra TED de Hans Rosling](https://www.youtube.com/watch?v=hVimVzgtD6w). Se você ainda não a assistiu, recomendamos que o faça primeiro!

Primeiro, vamos carregar o conjunto de dados da coleção [vega-datasets](https://github.com/vega/vega-datasets) em um _data frame_ do Pandas.

In [ ]:
from vega_datasets import data as vega_data
data = vega_data.gapminder()

Quão grande são os dados?

In [ ]:
data.shape

693 linhas e 6 colunas. Vamos dar uma olhada no conteúdo:

In [ ]:
data.head(5)


Para cada país (<span style="color:deeppink">country</span>) e ano (<span style="color:deeppink">year</span>) (em intervalos de 5 anos), temos medidas de números de crianças por mulher (<span style="color:deeppink">fertility</span>), espectativa de vida (<span style="color:deeppink">life_expect</span>) e total de população (<span style="color:deeppink">pop</span>).

Há também uma coluna chamada grupo (<span style="color:deeppink">cluster</span>) com códigos inteiros. Vamos descobrir o que isso representa ao longo da visualização!

Também vamos criar um _data frame_ de dados menor, filtrando somente os valores do anos 2000:

In [ ]:
data2000 = data.loc[data['year'] == 2000]

In [ ]:
data2000.head(5)

## Tipos de dados

O primeiro ingrediente para uma visualização eficaz são os dados de entrada. Os valores de dados podem representar diferentes formas de medição.
Que tipos de comparações essas medições suportam? E que tipos de codificações visuais apoiam essas comparações?

Começaremos analisando os tipos básicos de dados que o Altair usa para informar as escolhas de codificação visual. Esses tipos de dados determinam os tipos de comparações que podemos fazer e, assim, orientam nossas decisões no design de visualizações.



### Nominal (N)

Dados _nominais_ (também chamados de dados _categóricos_) consistem em nomes de categorias.

Com dados nominais, podemos comparar a igualdade dos valores: _o valor A é igual ou diferente do valor B? (A = B)_, permitindo afirmações como “A é igual a B” ou “A não é igual a B”. No conjunto de dados acima, o campo país (<span style="color:deeppink">country</span>) é nominal.

Ao visualizar dados nominais, devemos ser capazes de identificar facilmente se os valores são iguais ou diferentes: posição, matiz de cor (azul, vermelho, verde, etc.) e forma podem ajudar. No entanto, usar o canal de tamanho para codificar dados nominais pode nos enganar, sugerindo diferenças de ordem ou magnitude entre valores que não existem!

### Ordinal (O)

Dados _ordinais_ consistem em valores que possuem uma ordenação específica.

Com dados ordinais, podemos comparar a ordenação dos valores: _o valor A vem antes ou depois do valor B? (A < B)_, permitindo afirmações como “A é menor que B” ou “A é maior que B”. No conjunto de dados acima, podemos tratar o campo ano (<span style="color:deeppink">year</span>) como ordinal.

Ao visualizar dados ordinais, devemos perceber um senso de ordenação. Posição, tamanho ou valor da cor (brilho) podem ser apropriados, enquanto matiz de cor (que não é perceptualmente ordenada) seria menos adequado.


### Quantitativo (Q)

Com dados *quantitativos*, podemos medir diferenças numéricas entre valores. Existem vários subtipos de dados quantitativos:

Para dados *intervalares*, podemos medir a distância (intervalo) entre pontos: *qual é a distância do valor A ao valor B? (A - B)*, permitindo afirmações como “A está 12 unidades distante de B”.

Para dados *proporcionais*, o ponto zero é significativo, e assim podemos medir proporções ou fatores escalares:*o valor A corresponde a qual proporção do valor B? (A / B)*, permitindo afirmações como “A é 10% de B” ou “B é 7 vezes maior que A”.

No conjunto de dados acima, ano (<span style="color:deeppink">year</span>) é um campo quantitativo intervalar (o valor de “zero” no ano é subjetivo), enquanto fecundidade (<span style="color:deeppink">fertility</span>) e expectativa de vida (<span style="color:deeppink">life_expect</span>) são campos quantitativos proporcionais (zero é significativo para o cálculo de proporções). O Vega-Lite representa dados quantitativos, mas não faz distinção entre os tipos intervalares e proporcionais.

Valores quantitativos podem ser visualizados usando posição, tamanho ou valor da cor, entre outros canais. Um eixo com uma linha de base em zero é essencial para comparações proporcionais de valores proporcionais, mas pode ser omitido com segurança para comparações intervalares.


### Temporal (T)
Valores *temporais* medem pontos ou intervalos no tempo. Esse tipo é um caso especial de valores quantitativos (carimbos de data/hora) com semânticas e convenções ricas (por exemplo, o [Calendário Gregoriano](https://en.wikipedia.org/wiki/Gregorian_calendar)). O tipo temporal no Vega-Lite suporta raciocínio sobre unidades de tempo (ano, mês, dia, hora, etc.) e oferece métodos para solicitar intervalos de tempo específicos.

Exemplos de valores temporais incluem strings de datas como <span style="color:deeppink">“2019-01-04”</span> e <span style="color:deeppink">"Jan 04 2019"</span> , bem como formatos padronizados de data e hora, como o [Formato ISO](https://en.wikipedia.org/wiki/ISO_8601): <span style="color:deeppink">“2019-01-04T17:50:35.643Z”</span>.

Não há valores temporais em nosso conjunto de dados de desenvolvimento global acima, já que o campo ano (<span style="color:deeppink">year</span>) é simplesmente codificado como um número inteiro. Para mais detalhes sobre o uso de dados temporais no Altair, consulte a documentação de [Horas e Datas](https://altair-viz.github.io/user_guide/times_and_dates.html).

### Resumo

Estes tipos de dados não são mutuamente exclusivos. Na verdade, formam uma hierarquia: dados ordinais permitem comparação nominais (igualdades), enquanto dados quantitativos permitem comparações ordinais (ordenação).

Além disso, estes tipos de dados _não_ formam uma categorização única. Se um campo do dado é representado usando um número, isso não significa que devemos tratá-lo como um tipo quantitativo! Por exemplo, podemos interpretar um conjunto de idades (10 anos, 20 anos, etc) como um dado nominal (menor de idade ou maior de idade), ordinal (agrupado por ano), ou quantitativo (para o cálculo da média de idade).

Agora vamos analisar como codificar visualmente esses tipos de dados!

## Canais de Codificação

A utilização de *codificações* que associam *campos de dados* (contendo dados de um certo tipo) com *canais* de codificação (de um tipo da *marca* escolhido) está no coração do Altair. Nesta seção, vamos examinar os seguintes canais de codificação:

- <span style="color:deeppink">x</span>: Posição horizontal (eixo x) da marca.
- <span style="color:deeppink">y</span>: Posição vertical (eixo y) da marca.
- <span style="color:deeppink">size</span>: Tamanho da marca. Pode se referir a uma área ou comprimento, dependendo do tipo da marca.
- <span style="color:deeppink">color</span>: Determina a cor da marca, especificado como uma [cor válida CSS](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value).
- <span style="color:deeppink">opacity</span>: Determina a opacidade da marca, variando entre 0 (totalmente transparente) e 1 (totalmente opaco).
- <span style="color:deeppink">shape</span>: Determina o formato de marcas pontuais (<span style="color:deeppink">point</span>).
- <span style="color:deeppink">tooltip</span>: Informação textual a ser exibida ao passar o mouse sobre a marca.
- <span style="color:deeppink">order</span>: Determina a ordem. Estabelece a ordem de leitura e renderização de regiões da tela (linhas ou áreas).
- <span style="color:deeppink">column</span>: Organiza os dados em subgráficos alinhados horizontalmente.
- <span style="color:deeppink">row</span>: Organiza os dados em subgráficos alinhados verticalmente.

Veja a [documentação do Altair de codificação](https://altair-viz.github.io/user_guide/encoding.html) para uma lista completa dos canais disponíveis.

### X

O canal de codificação <span style="color:deeppink">x</span> determina a posição horizontal de um marcador (coordenada x). Além disso, são feitas automaticamente escolhas padrão para os eixos e título (caso não especificados). No gráfico abaixo, a escolha de um dado quantitativo resulta em um eixo contínuo com escala linear:

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q')
)

### Y

O canal de codificação <span style="color:deeppink">y</span> define a posição vertical (coordenada y). Aqui nós adicionamos o campo grupo (<span style="color:deeppink">cluster</span>) usando um tipo de dados ordinal (<span style="color:deeppink">o</span>). O resultado é um eixo discreto que inclui uma faixa com tamanho de passo padrão, para cada valor único:

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:O')
)

_O que acontece com o gráfico acima se você trocar os tipos de campo <span style="color:deeppink">O</span> e <span style="color:deeppink">Q</span>?_

Se, em vez disso, adicionarmos o campo expectativa de vida (<span style="color:deeppink">life_expect</span>) como uma variável quantitativa (<span style="color:deeppink">Q</span>), o resultado será um gráfico de dispersão com escalas lineares para ambos os eixos:

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q')
)

Por padrão, os eixos para escalas quantitativas lineares incluem o zero para garantir uma linha de base adequada para a comparação de razões de valores de dados. Em alguns casos, no entanto, uma linha de base zero pode não fazer sentido ou pode querer concentrar-se em comparações de intervalos. Para desativar a inclusão automática do zero, configure o mapeamento da escala utilizando o atributo de codificação escala (<span style="color:deeppink">scale</span>):

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q', scale=alt.Scale(zero=False)),
    alt.Y('life_expect:Q', scale=alt.Scale(zero=False))
)

Agora, as escalas dos eixos já não incluem o zero por padrão. Algum preenchimento ainda permanece, já que os pontos finais do domínio do eixo são automaticamente ajustados para números _amistosos_ (_nice_) como múltiplos de 5 ou 10.

_O que acontece se você também adicionar <span style="color:deeppink">(nice=false)</span> ao atributo scale acima?_

### Tamanho

O canal de codificação tamanho (<span style="color:deeppink">size</span>) define o tamanho ou a extensão de uma marca. O significado do canal pode variar de acordo com o tipo de marca. Para marcas <span style="color:deeppink">ponto</span>, o canal <span style="color:deeppink">size</span> mapeia para a área de pixel do símbolo de plotagem, de forma que o diâmetro do ponto corresponda à raiz quadrada do valor do tamanho.

Vamos aumentar o nosso gráfico de dispersão codificando a população (<span style="color:deeppink">pop</span>) no canal <span style="color:deeppink">size</span>. Como resultado, o gráfico agora também inclui uma legenda para interpretar os valores de tamanho.

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q')
)

Em alguns casos, podemos não estar satisfeitos com a faixa de tamanho padrão. Para fornecer um intervalo personalizado de tamanhos, defina o parâmetro intervalo (<span style="color:deeppink">range</span>) do atributo de escala (<span style="color:deeppink">scale</span>) como um intervalo que indica o tamanho mínimo e máximo. Aqui atualizamos a codificação de tamanho para variar de 0 pixels (para valores zero) a 1.000 pixels (para o valor máximo no domínio da escala):

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000]))
)

### Cor e Opacidade

O canal de codificação cor (<span style="color:deeppink">color</span>) define a cor de uma marca. O estilo de codificação da cor é altamente dependente do tipo de dados: os dados nominais terão, por padrão, um esquema de cores qualitativo multi-colorido, enquanto os dados ordinais e quantitativos utilizarão gradientes de  uma cor perceptualmente ordenados.

Aqui, codificamos o campo grupo (<span style="color:deeppink">cluster</span>) usando o canal _color_ e um tipo de dados nominal (<span style="color:deeppink">N</span>), resultando em uma tonalidade distinta para cada valor do grupo. Consegue começar a adivinhar o que o campo <span style="color:deeppink">cluster</span> pode indicar?

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N')
)

Se preferirmos formas preenchidas, podemos passar um parâmetro <span style="color:deeppink">filled=true</span> para o método <span style="color:deeppink">mark_point</span>:

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N')
)

Por padrão, o Altair usa um pouco de transparência para ajudar a combater o excesso de plotagem. Podemos ajustar ainda mais a opacidade, seja passando um valor padrão para o método <span style="color:deeppink">mark_*</span>, ou usando um canal de codificação dedicado.

Aqui demonstramos como fornecer um valor constante a um canal de codificação em vez de vincular um campo de dados:

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5)
)

### Formato

O canal de codificação formato (<span style="color:deeppink">shape</span>) define a forma geométrica usada pelos marcadores de <span style="color:deeppink">pontos</span>. Diferentemente dos outros canais que vimos até agora, o canal <span style="color:deeppink">shape</span> não pode ser usado por outros tipos de marca. O canal de codificação de forma deve ser usado apenas com dados nominais, pois comparações de ordem de classificação e magnitude perceptuais não são suportadas.

Vamos codificar o campo grupo (<span style="color:deeppink">cluster</span>) usando formato (<span style="color:deeppink">shape</span>), assim como cor (<span style="color:deeppink">color</span>). Usar vários canais para o mesmo campo de dados subjacente é conhecido como uma codificação redundante. O gráfico resultante combina informações de cor e forma em uma única legenda de símbolo:

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Shape('cluster:N')
)

### Dicas de ferramentas e ordenação

A essa altura, você pode se sentir um pouco frustrado: nós construímos um gráfico, mas ainda não sabemos a quais países os pontos visualizados correspondem! Vamos adicionar dicas de ferramentas interativas para permitir a exploração.

O canal de codificação de dicas de ferramenta (<span style="color:deeppink">tooltip</span>) determina o texto da dica de ferramenta a ser exibido quando um usuário move o cursos do mouse sobre uma marca. Vamos adicionar uma codificação de dica de ferramenta para o campo país (<span style="color:deeppink">country</span>) e, em seguida, investigar quais países estão sendo representados.

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Tooltip('country')
)

Conforme você passa o mouse, pode perceber que não consegue selecionar alguns pontos. Por exemplo, o maior círculo azul escuro corresponde a Índia, que é desenhado em cima de um país como uma população menor, impedindo que o mouse passe sobre esse país. Para corrigir esse problema, podemos usar o canal de codificação ordem (<span style="color:deeppink">order</span>).

O canal de codificação <span style="color:deeppink">order</span> determina a ordem dos pontos de dados, afetando tanto a ordem em que são desenhados quanto, para marcas linha (<span style="color:deeppink">line</span>) e área (<span style="color:deeppink">area</span>), a ordem em que são conectados uns aos outros.

Vamos ordenar os valores em ordem decrescente de classificação pela população (<span style="color:deeppink">pop</span>), garantindo que os círculos menores sejam desenhados depois dos círculos maiores:

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending')
)

Agora podemos identificar o país menor que está sendo obscurecido pela Índia: é Bangladesh!

Agora também podemos descobrir o que o campo grupo (<span style="color:deeppink">cluster</span>) representa. Passe o mouse sobre os vários pontos coloridos para formular sua própria explicação.

Neste ponto, adicionamos dicas de ferramentas que mostram apenas uma única propriedade do registro de dados subjacente. Para mostrar vários valores, podemos fornecer ao canal <span style="color:deeppink">tooltip</span> um array de codificações, uma para cada campo que queremos incluir: 

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Order('pop:Q', sort='descending'),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('fertility:Q'),
        alt.Tooltip('life_expect:Q')
    ]   
)

Agora podemos ver vários campos de dados ao passar o mouse!

<!--### Column and Row Facets-->

### Facetas de colunas e linhas 

<!--Spatial position is one of the most powerful and flexible channels for visual encoding, but what can we do if we already have assigned fields to the `x` and `y` channels? One valuable technique is to create a *trellis plot*, consisting of sub-plots that show a subset of the data. A trellis plot is one example of the more general technique of presenting data using [small multiples](https://en.wikipedia.org/wiki/Small_multiple) of views.

The `column` and `row` encoding channels generate either a horizontal (columns) or vertical (rows) set of sub-plots, in which the data is partitioned according to the provided data field.

Here is a trellis plot that divides the data into one column per \`cluster\` value:-->

<!--

    Algumas correções de tradução.

    Gráfico de Treliças -> Painel em grade
-->


A posição espacial é um dos tópicos mais poderosos e flexíveis para codificação visual de dados, mas o que podemos fazer se já atribuímos campos aos canais <span style="color:deeppink">x</span> e <span style="color:deeppink">y</span>? Uma técnica valiosa é criar um *painel em grade*, que consiste em subgráficos que mostram um subconjunto dos dados. Um painel em grade é um exemplo da técnica mais geral de apresentação de dados usando pequenos múltiplos ([Small multiple](https://en.wikipedia.org/wiki/Small_multiple)) de visualizações.

Os canais de codificação de coluna (<span style="color:deeppink">column</span>) e de linha (<span style="color:deeppink">line</span>) geram um conjunto horizontal (colunas) ou vertical (linhas) de subparcelas, nas quais os dados são particionados de acordo com o campo de dados fornecido.

Aqui está um painel em grade que divide os dados em uma coluna por valor de grupo (<span style="color:deeppink">cluster</span>):

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending'),
    alt.Column('cluster:N')
)

<!--The plot above does not fit on screen, making it difficult to compare all the sub-plots to each other! We can set the default `width` and `height` properties to create a smaller set of multiples. Also, as the column headers already label the `cluster` values, let's remove our `color` legend by setting it to `None`. To make better use of space we can also orient our `size` legend to the `'bottom'` of the chart.-->

A totalidade dos dados acima não cabe na tela, dificultando a comparação de todas as unidades gráficas entre si! Podemos definir as propriedades padrão  largura (<span style="color:deeppink">width</span>) e altura (<span style="color:deeppink">height</span>) para criar um conjunto menor de múltiplos. Além disso, como os cabeçalhos das colunas já rotulam os valores de grupo (<span style="color:deeppink">cluster</span>), vamos remover nossa legenda de cor (<span style="color:deeppink">color</span>) definindo-a como <span style="color:deeppink">None</span>. Para aproveitar melhor o espaço também podemos orientar nossa legenda de tamanho (<span style="color:deeppink">size</span>) para a parte inferior (<span style="color:deeppink">bottom</span>) do gráfico.

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000]),
             legend=alt.Legend(orient='bottom', titleOrient='left')),
    alt.Color('cluster:N', legend=None),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending'),
    alt.Column('cluster:N')
).properties(width=135, height=135)

<!--Underneath the hood, the `column` and `row` encodings are translated into a new specification that uses the `facet` view composition operator. We will re-visit faceting in greater depth later on!

In the meantime, _can you rewrite the chart above to facet into rows instead of columns?_-->

Implicitamente, as codificações <span style="color:deeppink">column</span> e <span style="color:deeppink">row</span> são traduzidas em uma nova especificação que usa o operador de composição de visualização faceta (<span style="color:deeppink">facet</span>). Iremos revisitar o facetamento com maior profundidade mais tarde!

Enquanto isso, _você consegue reescrever o gráfico acima para dividi-lo em linhas em vez de colunas?_

<!--### A Peek Ahead: Interactive Filtering

In later modules, we'll dive into interaction techniques for data exploration. Here is a sneak peak: binding a range slider to the `year` field to enable interactive scrubbing through each year of data. Don't worry if the code below is a bit confusing at this point, as we will cover interaction in detail later.

_Drag the slider back and forth to see how the data values change over time!_-->

### Uma prévia: filtragem interativa

Nos módulos posteriores, nos aprofundaremos nas técnicas de interação para exploração de dados. Aqui está uma prévia: vincular um controle deslizante de intervalo ao campo ano (<span style="color:deeppink">year</span>) para permitir a limpeza interativa de cada ano de dados. Não se preocupe se o código abaixo estiver um pouco confuso neste momento, pois abordaremos a interação em detalhes posteriormente.

_Arraste o controle deslizante para frente e para trás para ver como os valores dos dados mudam ao longo do tempo!_



In [ ]:
select_year = alt.selection_point(
    name='select',
    fields=['year'],
    value={'year': 1955},
    bind=alt.binding_range(min=1955, max=2005, step=5)
)

chart = alt.Chart(data).mark_point(filled=True).encode(
    alt.X('fertility:Q', scale=alt.Scale(domain=[0, 9])),
    alt.Y('life_expect:Q', scale=alt.Scale(domain=[0, 90])),
    alt.Size('pop:Q', scale=alt.Scale(domain=[0, 1200000000], range=[0, 1000])),
    alt.Color('cluster:N', legend=None),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending')
).add_params(
    select_year
).transform_filter(
    select_year
)
chart

<!--## Graphical Marks

Our exploration of encoding channels above exclusively uses `point` marks to visualize the data. However, the `point` mark type is only one of the many geometric shapes that can be used to visually represent data. Altair includes a number of built-in mark types, including:

- `mark_area()` - Filled areas defined by a top-line and a baseline.
- `mark_bar()` -	Rectangular bars.
- `mark_circle()`	- Scatter plot points as filled circles.
- `mark_line()` - Connected line segments.
- `mark_point()` - Scatter plot points with configurable shapes.
- `mark_rect()` - Filled rectangles, useful for heatmaps.
- `mark_rule()` - Vertical or horizontal lines spanning the axis.
- `mark_square()` - Scatter plot points as filled squares.
- `mark_text()` - Scatter plot points represented by text.
- `mark_tick()` - Vertical or horizontal tick marks.	

For a complete list, and links to examples, see the [Altair marks documentation](https://altair-viz.github.io/user_guide/marks.html). Next, we will step through a number of the most commonly used mark types for statistical graphics.-->

## Marcas Gráficas

Nossa exploração dos canais de codificação acima usa exclusivamente marcas do tipo ponto (<span style="color:deeppink">point</span>) para visualizar os dados. No entanto, o tipo de marca de ponto é apenas uma das muitas formas geométricas que podem ser usadas para representar dados visualmente. Altair inclui vários tipos de marcas integradas, incluindo:

- <span style="color:deeppink">mark_area()</span> - Áreas preenchidas definidas por uma linha superior e uma linha de base.
-  <span style="color:deeppink">mark_bar()</span> - Barras retangulares.
-  <span style="color:deeppink">mark_circle()</span> - Disperse os pontos do gráfico como círculos preenchidos.
-  <span style="color:deeppink">mark_line()</span> - Segmentos de linha conectados.
-  <span style="color:deeppink">mark_point()</span> - Disperse pontos do gráfico com formas configuráveis.
-  <span style="color:deeppink">mark_rect()</span> - Retângulos preenchidos, úteis para mapas de calor.
-  <span style="color:deeppink">mark_rule()</span> - Linhas verticais ou horizontais que abrangem o eixo.
-  <span style="color:deeppink">mark_square()</span> - Disperse os pontos do gráfico como quadrados preenchidos.
-  <span style="color:deeppink">mark_text()</span> - Pontos do gráfico de dispersão representados por texto.
-  <span style="color:deeppink">mark_tick()</span> - Marcas verticais ou horizontais.	

Para obter uma lista completa e links para exemplos, consulte a [documentação de marcas do Altair](https://altair-viz.github.io/user_guide/marks.html). A seguir, examinaremos alguns dos tipos de marcas mais comumente usados ​​para gráficos estatísticos.

### Marca de Ponto

O tipo marca de ponto (<span style="color:deeppink">point</span>) é utilizado para representar pontos específicos, como em gráficos de dispersão e gráficos de pontos. Além dos canais de codificação <span style="color:deeppink">x</span> e <span style="color:deeppink">y</span> (para especificar posições de pontos 2D), as marcas de ponto podem usar codificações de <span style="color:deeppink">color</span>,  <span style="color:deeppink">size</span> e <span style="color:deeppink">shape</span> para transmitir campos de dados adicionais.

Segue um gráfico de pontos da fecundidade (<span style="color:deeppink">fertility</span>), com o campo de grupo (<span style="color:deeppink">cluster</span>) codificado de forma redundante utilizando os canais <span style="color:deeppink">y</span> e de forma (<span style="color:deeppink">shape</span>).
 

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'), # fecundidade
    alt.Y('cluster:N'), # grupo
    alt.Shape('cluster:N') 
) 

Para além dos canais de codificação, as marcas podem ser estilizadas, basta fornecermos valores aos métodos <span style="color:deeppink">mark_*()</span>.

Por exemplo: as marcas de pontos são desenhadas com contornos traçados por predefinição, mas podem ser especificadas para utilizar formas preenchidas (<span style="color:deeppink">filled</span>). Da mesma forma, é possível predefinir um tamanho (<span style="color:deeppink">size</span>) para área total de pixels da marca de ponto.

In [ ]:
alt.Chart(data2000).mark_point(filled=True, size=100).encode(
    alt.X('fertility:Q'), # Fecundidade
    alt.Y('cluster:N'), # Grupo
    alt.Shape('cluster:N') 
)

### Marca de Círculo

O tipo marca de círculo (<span style="color:deeppink">circle</span>) é uma categoria conveniente para as marcas de ponto na forma de círculos preenchidos.

In [ ]:
alt.Chart(data2000).mark_circle(size=100).encode(
    alt.X('fertility:Q'), # Fecundidade
    alt.Y('cluster:N'), # Grupo
    alt.Shape('cluster:N')
)

### Marca de Quadrado

O tipo marca de quadrado (<span style="color:deeppink">square</span>) é uma categoria conveniente para as marcas de ponto na forma de quadrados preenchidos.

In [ ]:
alt.Chart(data2000).mark_square(size=100).encode(
    alt.X('fertility:Q'), # Fecundidade
    alt.Y('cluster:N'), # Grupo
    alt.Shape('cluster:N')
)

### Marca de Traço 

O tipo marca de traço (<span style="color:deeppink">tick</span>) transmite um ponto de dados utilizando um segmento de linha curto ou "tick". Estes são particularmente úteis para comparar valores ao longo de uma única dimensão com uma sobreposição mínima. Um gráfico de pontos desenhado com marcas de traço é por vezes referido como um *strip plot*.

In [ ]:
alt.Chart(data2000).mark_tick().encode(
    alt.X('fertility:Q'), # Fecundidade
    alt.Y('cluster:N'), # Grupo
    alt.Shape('cluster:N')
)

### Marcas de barra

O tipo marca de barra (<span style="color:deeppink">bar</span>) desenha um retângulo com uma posição, largura e altura.

O gráfico abaixo é um gráfico de barras simples da população (<span style="color:deeppink">pop</span>) de cada país.

In [ ]:
alt.Chart(data2000).mark_bar().encode(
    alt.X('country:N', title='País'),
    alt.Y('pop:Q', title='População')
)

A largura das barras é definida com um tamanho padrão. Discutiremos como ajustar a largura das barras mais adiante neste caderno. (Um caderno subsequente analisará mais de perto a configuração de eixos, escalas e legendas.)

As barras também podem ser empilhadas. Vamos alterar a codificação do eixo <span style="color:deeppink">x</span> para usar o campo de grupo (<span style="color:deeppink">cluster</span>) e codificar um país (<span style="color:deeppink">country</span>) utilizando o canal de cor (<span style="color:deeppink">color</span>). Também desativaremos a legenda (que seria muito longa com cores para todos os países!) e usaremos dicas de ferramenta (<span style="color:deeppink">tooltips</span>) para o nome do país.

In [ ]:
alt.Chart(data2000).mark_bar().encode(
    alt.X('cluster:N', title='Grupo'),
    alt.Y('pop:Q', title='População'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
)

No gráfico acima, o uso do canal de codificação de (<span style="color:deeppink">color</span>) faz com que o Altair / Vega-Lite empilhe automaticamente as marcas de barras. Caso contrário, as barras seriam desenhadas uma sobre a outra! Tente adicionar o parâmetro <span style="color:deeppink">stack=None</span> ao canal de codificação <span style="color:deeppink">y</span> para ver o que acontece se não aplicarmos o empilhamento...

Os exemplos acima criam gráficos de barras com a base partindo do zero, e o canal <span style="color:deeppink">y</span> codifica apenas a altura da barra. No entanto, a marca de barra também permite especificar pontos de início e fim para representar intervalos.

O gráfico abaixo usa os canais <span style="color:deeppink">x</span> (ponto de início) e <span style="color:deeppink">x2</span> (ponto de fim) para mostrar o intervalo de expectativa de vida dentro de cada grupo (<span style="color:deeppink">cluster</span>) regional. Abaixo, usamos as funções de agregação mínimo (<span style="color:deeppink">min</span>) e máximo (<span style="color:deeppink">max</span>) para determinar os pontos extremos do intervalo; discutiremos a agregação em mais detalhes no próximo _notebook_!

Alternativamente, você pode usar <span style="color:deeppink">x</span> e largura (<span style="color:deeppink">width</span>) para fornecer um ponto de início mais um deslocamento, de forma que <span style="color:deeppink">x2 = x + width</span>.

In [ ]:
alt.Chart(data2000).mark_bar().encode(
    alt.X('min(life_expect):Q', title='Mínimo de expectativa de vida'),
    alt.X2('max(life_expect):Q', title='Máximo de expectativa de vida'),
    alt.Y('cluster:N', title='Grupo')
)

### Marcas de linha

O tipo de marca de linha (<span style="color:deeppink">line</span>) conecta os pontos plotados com segmentos de linha para que, por exemplo, a inclinação da linha transmita informações sobre a taxa de variação.

Vamos exibir um gráfico de linha da fecundidade por país ao longo dos anos, usando o _data frame_ de desenvolvimento global. Novamente, vamos substituir a legenda por <span style="color:deeppink">_tooltips_</span>.

In [ ]:
alt.Chart(data).mark_line().encode(
    alt.X('year:O', title='Ano'),
    alt.Y('fertility:Q', title='Fecundidade'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
).properties(
    width=400
)


Podemos observar variações interessantes por país e tendências gerais de menores números de filhos por família ao longo do tempo. Também vale notar que definimos uma largura personalizada de 400 pixels. _Tente alterar (ou remover) as larguras e veja o que acontece!_

Vamos alterar alguns dos parâmetros padrão da marca para personalizar o gráfico. Podemos definir a largura do traço (<span style="color:deeppink">strokeWidth</span>) para determinar a espessura das linhas e a opacidade (<span style="color:deeppink">opacity</span>) para adicionar transparência. Por padrão, a marca  de linha (<span style="color:deeppink">line</span>) usa segmentos de linha reta para conectar os pontos de dados. Em alguns casos, podemos querer suavizar as linhas. Podemos ajustar a interpolação usada para conectar os pontos de dados configurando o parâmetro interpolação de marca (<span style="color:deeppink">interpolate</span>). Vamos usar a interpolação monótona (<span style="color:deeppink">monotone</span>) para fornecer linhas suaves que também garantem que não sejam gerados "falsos" valores mínimos ou máximos como resultado da interpolação.

In [ ]:
alt.Chart(data).mark_line(
    strokeWidth=3,
    opacity=0.5,
    interpolate='monotone'
).encode(
    alt.X('year:O', title='Ano'),
    alt.Y('fertility:Q', title='Fecundidade'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
).properties(
    width=400
)


A marca de <span style="color:deeppink">linha</span> também pode ser usada para criar *gráficos de inclinação* (slope graphs), ou seja, gráficos que destacam a mudança de valor entre dois pontos de comparação usando a inclinação das linhas.

Abaixo, vamos criar um gráfico de inclinação comparando as populações de cada país nos anos mínimo e máximo de nosso conjunto de dados: 1955 e 2005. Primeiro, criamos um novo _data frame_ do Pandas filtrado para esses anos e, em seguida, usamos o Altair para criar o gráfico de inclinação.

Por padrão, o Altair coloca os anos próximos uns dos outros. Para espaçar melhor os anos ao longo do eixo x, podemos indicar o tamanho (em pixels) dos passos discretos ao longo da largura do nosso gráfico, conforme indicado no comentário abaixo. Tente ajustar o valor de <span style="color:deeppink">step</span> abaixo e veja como o gráfico muda em resposta.

In [ ]:
dataTime = data.loc[(data['year'] == 1955) | (data['year'] == 2005)]

alt.Chart(dataTime).mark_line(opacity=0.5).encode(
    alt.X('year:O', title='Ano'),
    alt.Y('pop:Q', title='População'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
).properties(
    width={"step": 50} # ajusta o parâmetro de passos
)

### 2.4.7. Marcas de Área
A marca do tipo área (<span style="color:deeppink">area</span>) combina alguns aspectos da marca de linha (<span style="color:deeppink">line</span>) e de barra (<span style="color:deeppink">bar</span>): ela encontra conexões entre os pontos dos dados, e também exibe uma região preenchida, com uma borda correspondente a uma linha de base com valor nulo.


O gráfico abaixo é um gráfico de área com dados da população sobre o tempo dos Estados Unidos.

In [ ]:
dataUS = data.loc[data['country'] == 'United States']

alt.Chart(dataUS).mark_area().encode(
    alt.X('year:O'),
    alt.Y('fertility:Q')
)

Similarmente às marcas de linha, marcas de área  contam com um parâmetro de interpolação (<span style="color:deeppink">interpolate</span>).

In [ ]:
alt.Chart(dataUS).mark_area(interpolate='monotone').encode(
    alt.X('year:O'),
    alt.Y('fertility:Q')
)

Assim como marcas de barra, marcas de área também permitem o empilhamento (stacking). Aqui nós criamos um novo _data frame_ com os dados para os três países norte americanos e os plotamos utilizando a marca de área e um canal que indica a cor (<span style="color:deeppink">color</span>) para empilhar considerando o país.

In [ ]:
dataNA = data.loc[
    (data['country'] == 'United States') |
    (data['country'] == 'Canada') |
    (data['country'] == 'Mexico')
]

alt.Chart(dataNA).mark_area().encode(
    alt.X('year:O'),
    alt.Y('pop:Q'),
    alt.Color('country:N')
)

Por padrão, o _stacking_ é feito considerando uma linha de base nula, entretanto, temos outras opções disponíveis para fazê-lo:
- <span style="color:deeppink">center</span> - para empilhar em relação a uma linha de base no centro do gráfico, criando uma visualização como um gráfico de fluxo, e também
- <span style="color:deeppink">normalize</span> - para normalizar os dados como um todo em relação a 100%, assim, tornando possível comparações por porcentagem.


Abaixo, nós adaptamos o gráfico associando ao parâmetro <span style="color:deeppink">stack</span> no eixo <span style="color:deeppink">y</span>, o atributo <span style="color:deeppink">center</span>. O que aconteceria se colocássemos <span style="color:deeppink">normalize</span> ao invés de <span style="color:deeppink">center</span> no parâmetro?

In [ ]:
alt.Chart(dataNA).mark_area().encode(
    alt.X('year:O'),
    alt.Y('pop:Q', stack='center'),
    alt.Color('country:N')
)

Para desativar o stacking completamente, é necessário colocar o atributo do <span style="color:deeppink">stack</span> como <span style="color:deeppink">None</span>. Nós podemos também adicionar <span style="color:deeppink">opacity</span> como um parâmetro de marca padrão para assegurar que veremos as áreas de sobreposição!

In [ ]:
alt.Chart(dataNA).mark_area(opacity=0.5).encode(
    alt.X('year:O'),
    alt.Y('pop:Q', stack=None),
    alt.Color('country:N')
)

A marca de área também aceita linhas de base de dados direcionados, com ambas, séries superiores e inferiores determinadas pelo domínio dos dados. Assim como com marcas de barra, podemos usar os canais <span style="color:deeppink">x</span> e <span style="color:deeppink">x2</span> (ou <span style="color:deeppink">y</span> e <span style="color:deeppink">y2</span>) para obter pontos finais para as marcas de área.

O gráfico abaixo é uma visualização do intervalo de fecundidade máxima e mínima por ano para os países Norte Americanos:

In [ ]:
alt.Chart(dataNA).mark_area().encode(
    alt.X('year:O'),
    alt.Y('min(fertility):Q'),
    alt.Y2('max(fertility):Q')
).properties(
    width={"step": 40}
)

Podemos notar um maior intervalo de valores em 1995, de logo abaixo de 4 até logo abaixo de 7. Por volta de 2005, ambos os valores de fecundidade e variabilidade total tiveram uma queda, centrados em torno de 2 crianças por família.


Todos os exemplos de marca de área usados acima, usam um área verticalmente orientada. Entretanto, o Altair e o Vega-Lite também permitem áreas horizontais. Vamos transpor o gráfico acima simplesmente trocando os canais <span style="color:deeppink">x</span> e <span style="color:deeppink">y</span>

In [ ]:
alt.Chart(dataNA).mark_area().encode(
    alt.Y('year:O'),
    alt.X('min(fertility):Q'),
    alt.X2('max(fertility):Q')
).properties(
    width={"step": 40}
)

## Resumo

Concluímos nossa introdução aos tipos de dados, canais de codificação e marcas gráficas! Agora você deve estar bem preparado para explorar mais a fundo o espaço de codificações, tipos de marcas e parâmetros de marcas. Para uma referência abrangente – incluindo recursos que não abordamos aqui – consulte a documentação de [marcas](https://altair-viz.github.io/user_guide/marks.html) e [codificação](https://altair-viz.github.io/user_guide/encoding.html) do Altair.

No próximo módulo, veremos o uso de transformações de dados para criar gráficos que resumem dados ou visualizam novos campos derivados. Em um módulo futuro, examinaremos como personalizar ainda mais seus gráficos modificando escalas, eixos e legendas.

Interessado em aprender mais sobre codificação visual?
